## Deploying the agent to AgentCore Runtime

### Setup and Configuration

We'll start by setting up the Bedrock AgentCore starter toolkit

The [Bedrock AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit/) is a lightweight Python wrapper that streamlines deploying and managing AI agents on AWS Bedrock AgentCore. The toolkit acts as a thin layer over AWS Bedrock AgentCore's control and data plane APIs, automating infrastructure provisioning (ECR repositories, IAM roles, CodeBuild projects) and managing agent state through a .bedrock_agentcore.yaml configuration file. It provides both CLI and notebook interfaces that orchestrate the complete agent lifecycle - from initial configuration and containerization to cloud deployment and invocation. It supports multiple deployment modes (local development, cloud deployment with CodeBuild for ARM64 builds) and handles authentication patterns (JWT, OAuth, IAM) and protocol interfaces (HTTP, Model Context Protocol), enabling developers to focus on building agents rather than wrestling with AWS service integration details.

In [ ]:
import os
import random, string
from bedrock_agentcore_starter_toolkit import Runtime
import boto3
boto_session = boto3.Session()
region = boto_session.region_name


print(f"current region: {region}")
account_id = boto_session.client("sts").get_caller_identity()["Account"]
print(f"current account: {account_id}")

agentcore_runtime = Runtime()


### Create IAM Execution Role

The AgentCore runtime requires an IAM role with specific permissions to:
- Access Bedrock models for AI inference
- Retrieve from knowledge bases (KB)
- Write to CloudWatch logs
- Access SSM Parameter for KB ID
- Pull/push Docker images to ECR

We'll create this role using our predefined policy templates that include all necessary permissions.

In [ ]:
# Load policy from external file
with open('policy.json', 'r') as f:
    policy_template = f.read()

# load trust policy from external file
with open('trust-policy.json', 'r') as f:
    trust_policy_template = f.read()

policy = policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id).replace('REPO_ARN', '*')
print(f"Policy loaded and updated with region: {region}, account: {account_id}")

trust_policy = trust_policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id)

# create IAM role using the policies
suffix = random.choices(string.ascii_lowercase + string.digits, k=8)
iam_client = boto3.client('iam')
role_name = f"bedrock-runtime-execution-role-{''.join(suffix)}"

role = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=trust_policy
)

iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName='bedrock-runtime-execution-policy',
    PolicyDocument=policy
)

### Configure Agent Deployment

The `configure()` method sets up the deployment infrastructure:


**What happens during configuration:**
- Creates ECR repository and IAM role (if needed)
- Generates Dockerfile and .dockerignore for containerization
- Creates `.bedrock_agentcore.yaml` configuration file

In [ ]:
agent_name = "mortgage_assistant"
response = agentcore_runtime.configure(
    entrypoint="mortgage_agent_runtime.py",
    execution_role=role['Role']['Arn'],
    # auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### Launch Agent Runtime

The `launch()` method triggers a CodeBuild job that:

1. **Pre-build**: Authenticates with ECR registry
2. **Build**: Creates ARM64 Docker image with your agent code
3. **Post-build**: Pushes image to ECR and creates a Bedrock AgentCore runtime and deploys the agent

In [ ]:
launch_result = agentcore_runtime.launch()

Lets check the status of the agentcore runtime

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(status)

Time to invoke the agent

In [ ]:
import json

def print_response_text(invoke_response):
    response_bytes=invoke_response['response']
    response_strings = [item.decode('utf-8') for item in response_bytes]
    # print(response_bytes)
    response_combined_string = "".join(response_strings)
    # print(response_combined_string)
    
    response_dict = json.loads(response_combined_string)
    response_text=response_dict["result"]["content"][0]["text"]
    print(response_text)

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "what is my current outstanding balance for customer id 1234??"})
print_response_text(invoke_response)

## Summary

You've successfully deployed a multi-agent mortgage assistant that:

- **Routes customers** to appropriate specialists automatically  
- **Retrieves real-time data** using  tools  
- **Accesses knowledge base** for general mortgage guidance  
- **Handles complex queries** requiring multiple agent expertise  



## Related Resources

- [Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock/)
- [Strands Documentation](https://strandsagents.com/latest/documentation/docs/)